#### Link: https://towardsdatascience.com/pyspark-and-sparksql-basics-6cb4bf967e53

In [1]:
import findspark

findspark.init('C:\spark\spark-2.4.5-bin-hadoop2.7/')

In [2]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import * 
from pyspark.sql.types import * 
from datetime import date, timedelta, datetime
import time

In [3]:
# Initializing SparkSession
sc = SparkSession.builder.appName("PysparkExample")\
.config ("spark.sql.shuffle.partitions", "50").config("spark.driver.maxResultSize","5g")\
.config ("spark.sql.execution.arrow.enabled", "true").getOrCreate()

In [6]:
# Creating Data Frames
#Creates a spark data frame called as raw_data.
#JSON
dataframe = sc.read.json('data/nyt2.json')

#TXT FILES# 
#dataframe_txt = sc.read.text('text_data.txt')

#CSV FILES# 
#dataframe_csv = sc.read.csv('csv_data.csv')

#PARQUET FILES# 
#dataframe_parquet = sc.read.load('parquet_data.parquet')

In [9]:
dataframe

DataFrame[_id: struct<$oid:string>, amazon_product_url: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>]

In [10]:
dataframe.show(10)

+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...|       Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|
|[5b4aa4ead3089013...|http://www.amazon...|     Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|Little, Brown| [2]|           [1]|            THE HOST|          [3]|


In [11]:
dataframe_dropdup = dataframe.dropDuplicates() 
dataframe_dropdup.show(10)

+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+----------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|   price|   published_date|       publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+----------------+----+--------------+--------------------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...|Clive Cussler wit...|[[1213401600000]]|Juan Cabrillo and...|[26.95,]|[[1214697600000]]|          Putnam| [4]|           [3]|         PLAGUE SHIP|          [2]|
|[5b4aa4ead3089013...|http://www.amazon...|      Jeffery Deaver|[[1215820800000]]|Detectives Lincol...|   [, 0]|[[1217116800000]]|Simon & Schuster|[20]|           [0]|   THE BROKEN WINDOW|

In [12]:
# “Select” Operation
#Show all entries in title column
dataframe.select("author").show(10)

+--------------------+
|              author|
+--------------------+
|       Dean R Koontz|
|     Stephenie Meyer|
|        Emily Giffin|
|   Patricia Cornwell|
|     Chuck Palahniuk|
|James Patterson a...|
|       John Sandford|
|       Jimmy Buffett|
|    Elizabeth George|
|      David Baldacci|
+--------------------+
only showing top 10 rows



In [13]:
#Show all entries in title, author, rank, price columns
dataframe.select("author", "title", "rank", "price").show(10)

+--------------------+--------------------+----+--------+
|              author|               title|rank|   price|
+--------------------+--------------------+----+--------+
|       Dean R Koontz|           ODD HOURS| [1]|  [, 27]|
|     Stephenie Meyer|            THE HOST| [2]|[25.99,]|
|        Emily Giffin|LOVE THE ONE YOU'...| [3]|[24.95,]|
|   Patricia Cornwell|           THE FRONT| [4]|[22.95,]|
|     Chuck Palahniuk|               SNUFF| [5]|[24.95,]|
|James Patterson a...|SUNDAYS AT TIFFANY’S| [6]|[24.99,]|
|       John Sandford|        PHANTOM PREY| [7]|[26.95,]|
|       Jimmy Buffett|          SWINE NOT?| [8]|[21.99,]|
|    Elizabeth George|     CARELESS IN RED| [9]|[27.95,]|
|      David Baldacci|     THE WHOLE TRUTH|[10]|[26.99,]|
+--------------------+--------------------+----+--------+
only showing top 10 rows



In [16]:
# “When” Operation
# Show title and assign 0 or 1 depending on title
dataframe.select("title",when(dataframe.title != 'ODD HOURS', 
1).otherwise(0)).show(10)

+--------------------+-----------------------------------------------------+
|               title|CASE WHEN (NOT (title = ODD HOURS)) THEN 1 ELSE 0 END|
+--------------------+-----------------------------------------------------+
|           ODD HOURS|                                                    0|
|            THE HOST|                                                    1|
|LOVE THE ONE YOU'...|                                                    1|
|           THE FRONT|                                                    1|
|               SNUFF|                                                    1|
|SUNDAYS AT TIFFANY’S|                                                    1|
|        PHANTOM PREY|                                                    1|
|          SWINE NOT?|                                                    1|
|     CARELESS IN RED|                                                    1|
|     THE WHOLE TRUTH|                                                    1|

In [18]:
# Show rows with specified authors if in the given options
dataframe[dataframe.author.isin("John Sandford", "Emily Giffin")].show(5)

+--------------------+--------------------+-------------+-----------------+--------------------+--------+-----------------+------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|       author| bestsellers_date|         description|   price|   published_date|   publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+-------------+-----------------+--------------------+--------+-----------------+------------+----+--------------+--------------------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...| Emily Giffin|[[1211587200000]]|A woman's happy m...|[24.95,]|[[1212883200000]]|St. Martin's| [3]|           [2]|LOVE THE ONE YOU'...|          [2]|
|[5b4aa4ead3089013...|http://www.amazon...|John Sandford|[[1211587200000]]|The Minneapolis d...|[26.95,]|[[1212883200000]]|      Putnam| [7]|           [4]|        PHANTOM PREY|          [3]|
|[5b4aa4ead3089013...|http://www.amazon.

In [19]:
# “Like” Operation
# Show author and title is TRUE if title has " THE " word in titles
dataframe.select("author", "title", dataframe.title.like("% THE %")).show(15)

+--------------------+--------------------+------------------+
|              author|               title|title LIKE % THE %|
+--------------------+--------------------+------------------+
|       Dean R Koontz|           ODD HOURS|             false|
|     Stephenie Meyer|            THE HOST|             false|
|        Emily Giffin|LOVE THE ONE YOU'...|              true|
|   Patricia Cornwell|           THE FRONT|             false|
|     Chuck Palahniuk|               SNUFF|             false|
|James Patterson a...|SUNDAYS AT TIFFANY’S|             false|
|       John Sandford|        PHANTOM PREY|             false|
|       Jimmy Buffett|          SWINE NOT?|             false|
|    Elizabeth George|     CARELESS IN RED|             false|
|      David Baldacci|     THE WHOLE TRUTH|             false|
|        Troy Denning|          INVINCIBLE|             false|
|          James Frey|BRIGHT SHINY MORNING|             false|
|         Garth Stein|THE ART OF RACING...|            

In [20]:
# “Startswith” — “ Endswith”
dataframe.select("author", "title", dataframe.title.startswith("THE")).show(5)

+-----------------+--------------------+----------------------+
|           author|               title|startswith(title, THE)|
+-----------------+--------------------+----------------------+
|    Dean R Koontz|           ODD HOURS|                 false|
|  Stephenie Meyer|            THE HOST|                  true|
|     Emily Giffin|LOVE THE ONE YOU'...|                 false|
|Patricia Cornwell|           THE FRONT|                  true|
|  Chuck Palahniuk|               SNUFF|                 false|
+-----------------+--------------------+----------------------+
only showing top 5 rows



In [21]:
dataframe.select("author", "title", dataframe.title.endswith("NT")).show(5)

+-----------------+--------------------+-------------------+
|           author|               title|endswith(title, NT)|
+-----------------+--------------------+-------------------+
|    Dean R Koontz|           ODD HOURS|              false|
|  Stephenie Meyer|            THE HOST|              false|
|     Emily Giffin|LOVE THE ONE YOU'...|              false|
|Patricia Cornwell|           THE FRONT|               true|
|  Chuck Palahniuk|               SNUFF|              false|
+-----------------+--------------------+-------------------+
only showing top 5 rows



In [22]:
# “Substring” Operation
dataframe.select(dataframe.author.substr(1, 3).alias("title")).show(5)

+-----+
|title|
+-----+
|  Dea|
|  Ste|
|  Emi|
|  Pat|
|  Chu|
+-----+
only showing top 5 rows



In [23]:
dataframe.select(dataframe.author.substr(3, 6).alias("title")).show(5)

+------+
| title|
+------+
|an R K|
|epheni|
|ily Gi|
|tricia|
|uck Pa|
+------+
only showing top 5 rows



In [24]:
dataframe.select(dataframe.author.substr(1, 6).alias("title")).show(5)

+------+
| title|
+------+
|Dean R|
|Stephe|
|Emily |
|Patric|
|Chuck |
+------+
only showing top 5 rows



In [26]:
# Adding Columns
# Lit() is required while we are creating columns with exact values.
dataframe = dataframe.withColumn('new_column', lit('This is a new column'))
display(dataframe)

DataFrame[_id: struct<$oid:string>, amazon_product_url: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>, new_column: string]

In [27]:
# Updating Columns
# Update column 'amazon_product_url' with 'URL'
dataframe = dataframe.withColumnRenamed('amazon_product_url', 'URL')
dataframe.show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|L

In [28]:
# Removing Columns
dataframe_remove = dataframe.drop("publisher", "published_date").show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|   price|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]| [2]|           [1]|            THE HOST|          [3]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|     Emily Giffin|[[1211587

In [29]:
dataframe_remove2 = dataframe.drop(dataframe.publisher).drop(dataframe.published_date).show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|   price|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]| [2]|           [1]|            THE HOST|          [3]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|     Emily Giffin|[[1211587

### Inspect Data

In [30]:
# Returns dataframe column names and data types
dataframe.dtypes

[('_id', 'struct<$oid:string>'),
 ('URL', 'string'),
 ('author', 'string'),
 ('bestsellers_date', 'struct<$date:struct<$numberLong:string>>'),
 ('description', 'string'),
 ('price', 'struct<$numberDouble:string,$numberInt:string>'),
 ('published_date', 'struct<$date:struct<$numberLong:string>>'),
 ('publisher', 'string'),
 ('rank', 'struct<$numberInt:string>'),
 ('rank_last_week', 'struct<$numberInt:string>'),
 ('title', 'string'),
 ('weeks_on_list', 'struct<$numberInt:string>'),
 ('new_column', 'string')]

In [31]:
# Displays the content of dataframe
dataframe.show()

+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+--------------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|              author| bestsellers_date|         description|   price|   published_date|           publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+--------------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|       Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|              Bantam| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|     Stephenie Meyer|[[1211587200000]]|Aliens

In [35]:
# Return first n rows
dataframe.head()

Row(_id=Row($oid='5b4aa4ead3089013507db18b'), URL='http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20', author='Dean R Koontz', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.', price=Row($numberDouble=None, $numberInt='27'), published_date=Row($date=Row($numberLong='1212883200000')), publisher='Bantam', rank=Row($numberInt='1'), rank_last_week=Row($numberInt='0'), title='ODD HOURS', weeks_on_list=Row($numberInt='1'), new_column='This is a new column')

In [36]:
# Returns first row
dataframe.first()

Row(_id=Row($oid='5b4aa4ead3089013507db18b'), URL='http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20', author='Dean R Koontz', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.', price=Row($numberDouble=None, $numberInt='27'), published_date=Row($date=Row($numberLong='1212883200000')), publisher='Bantam', rank=Row($numberInt='1'), rank_last_week=Row($numberInt='0'), title='ODD HOURS', weeks_on_list=Row($numberInt='1'), new_column='This is a new column')

In [37]:
# Return first n rows
dataframe.take(5)

[Row(_id=Row($oid='5b4aa4ead3089013507db18b'), URL='http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20', author='Dean R Koontz', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.', price=Row($numberDouble=None, $numberInt='27'), published_date=Row($date=Row($numberLong='1212883200000')), publisher='Bantam', rank=Row($numberInt='1'), rank_last_week=Row($numberInt='0'), title='ODD HOURS', weeks_on_list=Row($numberInt='1'), new_column='This is a new column'),
 Row(_id=Row($oid='5b4aa4ead3089013507db18c'), URL='http://www.amazon.com/The-Host-Novel-Stephenie-Meyer/dp/0316218502?tag=NYTBS-20', author='Stephenie Meyer', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Aliens have taken control of the minds and bodies of most humans, but one woman won’t surrender.', price=Row($numberDouble='25.99', $numberInt=None), published_

In [38]:
# Computes summary statistics
dataframe.describe().show()

+-------+--------------------+---------------+--------------------+---------+------------------+--------------------+
|summary|                 URL|         author|         description|publisher|             title|          new_column|
+-------+--------------------+---------------+--------------------+---------+------------------+--------------------+
|  count|               10195|          10195|               10195|    10195|             10195|               10195|
|   mean|                null|           null|                null|     null|1877.7142857142858|                null|
| stddev|                null|           null|                null|     null| 370.9760613506458|                null|
|    min|http://www.amazon...|        AJ Finn|                    |      ACE|  10TH ANNIVERSARY|This is a new column|
|    max|https://www.amazo...|various authors|’Tis for the Rebe...|allantine|               ZOO|This is a new column|
+-------+--------------------+---------------+----------

In [39]:
# Returns columns of dataframe
dataframe.columns

['_id',
 'URL',
 'author',
 'bestsellers_date',
 'description',
 'price',
 'published_date',
 'publisher',
 'rank',
 'rank_last_week',
 'title',
 'weeks_on_list',
 'new_column']

In [40]:
# Counts the number of rows in dataframe
dataframe.count()

10195

In [41]:
# Counts the number of distinct rows in dataframe
dataframe.distinct().count()

10195

In [42]:
# Prints plans including physical and logical
dataframe.explain(4)

== Parsed Logical Plan ==
Project [_id#6, amazon_product_url#7 AS URL#358, author#8, bestsellers_date#9, description#10, price#11, published_date#12, publisher#13, rank#14, rank_last_week#15, title#16, weeks_on_list#17, new_column#344]
+- Project [_id#6, amazon_product_url#7, author#8, bestsellers_date#9, description#10, price#11, published_date#12, publisher#13, rank#14, rank_last_week#15, title#16, weeks_on_list#17, This is a new column AS new_column#344]
   +- Relation[_id#6,amazon_product_url#7,author#8,bestsellers_date#9,description#10,price#11,published_date#12,publisher#13,rank#14,rank_last_week#15,title#16,weeks_on_list#17] json

== Analyzed Logical Plan ==
_id: struct<$oid:string>, URL: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$n

### 8. “GroupBy” Operation

In [5]:
# Group by author, count the books of the authors in the groups
dataframe.groupBy("author").count().show(10)

+--------------------+-----+
|              author|count|
+--------------------+-----+
|          James Frey|    2|
|    Elin Hilderbrand|   58|
|   Sharon Kay Penman|    2|
|         Kate Jacobs|    3|
|       Karen Robards|    6|
|     Gary Shteyngart|    3|
|         Lisa Genova|    7|
|James Patterson a...|   30|
|         Ruth Reichl|    3|
|         JRR Tolkien|    2|
+--------------------+-----+
only showing top 10 rows



### 9. “Filter” Operation

In [6]:
# Filtering entries of title
# Only keeps records having value 'THE HOST'
dataframe.filter(dataframe["title"] == 'THE HOST').show(5)

+--------------------+--------------------+---------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------+-------------+
|                 _id|  amazon_product_url|         author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|   title|weeks_on_list|
+--------------------+--------------------+---------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|Little, Brown| [2]|           [1]|THE HOST|          [3]|
|[5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|[[1212192000000]]|Aliens have taken...|[25.99,]|[[1213488000000]]|Little, Brown| [2]|           [2]|THE HOST|          [4]|
|[5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|[[1212796800000]]|Aliens h

### 10. Missing & Replacing Values

In [10]:
# Replacing null values
dataframe.na.fill(0).show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|           author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|Little, Brown| [2]|           [1]|            THE HOST|          [3]|
|[5b4aa4ead3089

In [12]:
dataframe.fillna(0).show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|           author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|Little, Brown| [2]|           [1]|            THE HOST|          [3]|
|[5b4aa4ead3089

In [14]:
dataFrameNaFunctions.fill()

NameError: name 'dataFrameNaFunctions' is not defined

In [17]:
# Returning new dataframe restricting rows with null 
dataframe.na.drop().show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|           author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|Little, Brown| [2]|           [1]|            THE HOST|          [3]|
|[5b4aa4ead3089

In [18]:
dataframe.dropna().show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|           author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|Little, Brown| [2]|           [1]|            THE HOST|          [3]|
|[5b4aa4ead3089

In [ ]:
dataFrameNaFunctions.drop()

In [ ]:
# Return new dataframe replacing one value with another
dataframe.na.replace(5, 15)
dataFrame.replace()
dataFrameNaFunctions.replace()

### 11. Repartitioning

In [7]:
# Number of partitions
dataframe.rdd.getNumPartitions()

2

In [18]:
# Dataframe with 10 partitions
dataframe.repartition(10).rdd.getNumPartitions()

10

In [24]:
# Dataframe with 1 partition
dataframe.coalesce(1).rdd.getNumPartitions()

1

### 12. Running SQL Queries Programmatically

In [25]:
# Registering a table
dataframe.registerTempTable("table_1")

In [26]:
sc.sql("select * from table_1").show(3)

+--------------------+--------------------+---------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|         author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+---------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...|  Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|
|[5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|Little, Brown| [2]|           [1]|            THE HOST|          [3]|
|[5b4aa4ead3089013...|htt

In [29]:
sc.sql("""select 
          CASE WHEN description LIKE '%love%' THEN 'Love_Theme' 
          WHEN description LIKE '%hate%' THEN 'Hate_Theme'
          WHEN description LIKE '%happy%' THEN 'Happiness_Theme' 
          WHEN description LIKE '%anger%' THEN 'Anger_Theme'
          WHEN description LIKE '%horror%' THEN 'Horror_Theme'
          WHEN description LIKE '%death%' THEN 'Criminal_Theme'
          WHEN description LIKE '%detective%' THEN 'Mystery_Theme'
          ELSE 'Other_Themes'
          END Themes
          from table_1
          """).registerTempTable("table_2")

In [32]:
sc.sql("""select * from table_2""").groupBy('Themes').count().show()
#.groupBy('Themes').count().show()

+---------------+-----+
|         Themes|count|
+---------------+-----+
|    Anger_Theme|  203|
|   Other_Themes| 8778|
|  Mystery_Theme|  454|
|     Hate_Theme|   23|
| Criminal_Theme|  305|
|   Horror_Theme|    6|
|Happiness_Theme|   34|
|     Love_Theme|  392|
+---------------+-----+



In [34]:
sc.sql("""select Themes, count(*) from table_2
          group by Themes
       """).show()

+---------------+--------+
|         Themes|count(1)|
+---------------+--------+
|    Anger_Theme|     203|
|   Other_Themes|    8778|
|  Mystery_Theme|     454|
|     Hate_Theme|      23|
| Criminal_Theme|     305|
|   Horror_Theme|       6|
|Happiness_Theme|      34|
|     Love_Theme|     392|
+---------------+--------+



### 13. Output

In [ ]:
# 13.1. Data Structures

In [35]:
# Converting dataframe into an RDD
rdd_convert = dataframe.rdd

In [36]:
# Converting dataframe into a RDD of string 
dataframe.toJSON().first()

'{"_id":{"$oid":"5b4aa4ead3089013507db18b"},"amazon_product_url":"http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20","author":"Dean R Koontz","bestsellers_date":{"$date":{"$numberLong":"1211587200000"}},"description":"Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.","price":{"$numberInt":"27"},"published_date":{"$date":{"$numberLong":"1212883200000"}},"publisher":"Bantam","rank":{"$numberInt":"1"},"rank_last_week":{"$numberInt":"0"},"title":"ODD HOURS","weeks_on_list":{"$numberInt":"1"}}'

In [37]:
dataframe.show(1)

+--------------------+--------------------+-------------+-----------------+--------------------+------+-----------------+---------+----+--------------+---------+-------------+
|                 _id|  amazon_product_url|       author| bestsellers_date|         description| price|   published_date|publisher|rank|rank_last_week|    title|weeks_on_list|
+--------------------+--------------------+-------------+-----------------+--------------------+------+-----------------+---------+----+--------------+---------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...|Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|[, 27]|[[1212883200000]]|   Bantam| [1]|           [0]|ODD HOURS|          [1]|
+--------------------+--------------------+-------------+-----------------+--------------------+------+-----------------+---------+----+--------------+---------+-------------+
only showing top 1 row



In [39]:
# Obtaining contents of df as Pandas 
dataframe.toPandas()

C:\spark\spark-2.4.5-bin-hadoop2.7/python\pyspark\sql\dataframe.py:2110: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 0.8.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


,_id,amazon_product_url,author,bestsellers_date,description,price,published_date,publisher,rank,rank_last_week,title,weeks_on_list
0,"(5b4aa4ead3089013507db18b,)",http://www.amazon.com/Odd-Hours-Dean-Koontz/dp...,Dean R Koontz,"((1211587200000,),)","Odd Thomas, who can communicate with the dead,...","(None, 27)","((1212883200000,),)",Bantam,"(1,)","(0,)",ODD HOURS,"(1,)"
1,"(5b4aa4ead3089013507db18c,)",http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,"((1211587200000,),)",Aliens have taken control of the minds and bod...,"(25.99, None)","((1212883200000,),)","Little, Brown","(2,)","(1,)",THE HOST,"(3,)"
2,"(5b4aa4ead3089013507db18d,)",http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,"((1211587200000,),)",A woman's happy marriage is shaken when she en...,"(24.95, None)","((1212883200000,),)",St. Martin's,"(3,)","(2,)",LOVE THE ONE YOU'RE WITH,"(2,)"
3,"(5b4aa4ead3089013507db18e,)",http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,"((1211587200000,),)",A Massachusetts state investigator and his tea...,"(22.95, None)","((1212883200000,),)",Putnam,"(4,)","(0,)",THE FRONT,"(1,)"
4,"(5b4aa4ead3089013507db18f,)",http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,"((1211587200000,),)",An aging porn queens aims to cap her career by...,"(24.95, None)","((1212883200000,),)",Doubleday,"(5,)","(0,)",SNUFF,"(1,)"
...,...,...,...,...,...,...,...,...,...,...,...,...
10190,"(5b4aa4ead3089013507dd959,)",https://www.amazon.com/Clancy-Line-Sight-Jack-...,Mike Maden,"((1530921600000,),)",Jack Ryan Jr. risks his life to protect a woma...,"(None, 0)","((1532217600000,),)",Putnam,"(11,)","(6,)",TOM CLANCY LINE OF SIGHT,"(4,)"
10191,"(5b4aa4ead3089013507dd95a,)",https://www.amazon.com/Something-Water-Novel-C...,Catherine Steadman,"((1530921600000,),)",A documentary filmmaker and an investment bank...,"(None, 0)","((1532217600000,),)",Ballantine,"(12,)","(11,)",SOMETHING IN THE WATER,"(5,)"
10192,"(5b4aa4ead3089013507dd95b,)",https://www.amazon.com/Little-Fires-Everywhere...,Celeste Ng,"((1530921600000,),)",An artist upends a quiet town outside Cleveland.,"(None, 0)","((1532217600000,),)",Penguin Press,"(13,)","(12,)",LITTLE FIRES EVERYWHERE,"(41,)"
10193,"(5b4aa4ead3089013507dd95c,)",https://www.amazon.com/Shelter-Place-Nora-Robe...,Nora Roberts,"((1530921600000,),)",Survivors of a mass shooting outside a mall in...,"(None, 0)","((1532217600000,),)",St. Martin's,"(14,)","(5,)",SHELTER IN PLACE,"(6,)"


In [ ]:
# 13.2. Write & Save to Files

In [40]:
# Write & Save File in .parquet format
dataframe.select("author", "title", "rank", "description") \
.write \
.save("data/Rankings_Descriptions.parquet")

In [41]:
# Write & Save File in .json format
dataframe.select("author", "title") \
.write \
.save("data/Authors_Titles.json",format="json")

In [42]:
# End Spark Session
sc.stop()